In [2]:
#Build a model that takes human images and then classisifes them according to the emotion detected

In [83]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.utils import to_categorical
import os
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt
%matplotlib inline
from keras.applications import vgg16
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint

In [4]:
from keras.applications import VGG16

vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [80]:
print(vgg_conv.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

train_dir = './clean-dataset/train'
validation_dir = './clean-dataset/validation'

nTrain = 1040 
nVal = 

In [95]:
train_dir = 'C:/Users/RAHUL/Downloads/images/train'
validation_dir = 'C:/Users/RAHUL/Downloads/images/test'

nTrain = 1560 #260 x 6
nVal = 180 #30 x 6

In [7]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

train_features = np.zeros(shape=(nTrain, 7, 7, 512))
train_labels = np.zeros(shape=(nTrain,6))
#I HAVE 6 classes now

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain:
        break
        
train_features = np.reshape(train_features, (nTrain, 7 * 7 * 512))

Found 1560 images belonging to 6 classes.


In [96]:
validation_features = np.zeros(shape=(nVal, 7, 7, 512))
validation_labels = np.zeros(shape=(nVal,6))
# 6 here as well

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

i = 0
for inputs_batch, labels_batch in validation_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    validation_features[i * batch_size : (i + 1) * batch_size] = features_batch
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nVal:
        break

validation_features = np.reshape(validation_features, (nVal, 7 * 7 * 512))

Found 240 images belonging to 6 classes.


In [97]:
save_path='C:/Users/RAHUL/Downloads/images/'

In [98]:
from keras import models
from keras import layers
from keras import optimizers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(512, activation='relu', input_dim=7 * 7 * 512))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(6, activation='softmax'))

    model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
                  loss='categorical_crossentropy',
                  metrics=['acc'])
    print(model.summary())
    return model
def train_model(model,reload_filename='NULL',save_best_only=True,batch_size=20,num_epochs=10):
    if(save_best_only):
        checkpointer = ModelCheckpoint(filepath=save_path + '/model-{epoch:02d}-{loss:.4f}.hdf5', verbose=1,monitor='val_acc', save_best_only=True, mode='auto')
    else:
        checkpointer = ModelCheckpoint(filepath=save_path + '/model-{epoch:02d}-{loss:.4f}.hdf5', verbose=1)
    #period option in checkpointer -> after how many epochs to save the model
    if(reload_filename!="NULL"):
            model = load_model(save_path + reload_filename)
    model.fit(train_features,
                    train_labels,
                    epochs=num_epochs,
                    batch_size=20,
                    validation_data=(validation_features,validation_labels), callbacks=[checkpointer])
    model.save(save_path + "final_model.hdf5")

In [99]:
model=build_model()
reload_filename="final_emotion_model.hdf5"
train_model(model, reload_filename=reload_filename)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 512)               12845568  
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 6)                 3078      
Total params: 12,848,646
Trainable params: 12,848,646
Non-trainable params: 0
_________________________________________________________________
None
Train on 1560 samples, validate on 180 samples
Epoch 1/10
1560/1560 [==============================] - 22s 14ms/step - loss: 0.1297 - acc: 0.9571 - val_loss: 2.3522 - val_acc: 0.4389

Epoch 00001: val_acc improved from -inf to 0.43889, saving model to C:/Users/RAHUL/Downloads/images//model-01-0.1297.hdf5
Epoch 2/10
1560/1560 [==============================] - 20s 13ms/step - loss: 0.123